In [1]:
from fairlets.MCF_decomposition import MCF_decomposition
from fairlets.vanilla_decomposition import Vanilla_decomposition
from helper.data_loader import data_loader
from helper.k_center import k_center
#from helper.plot_helper import plotHelper
import numpy as np

In [2]:
from experiments import FairletExperiment

In [3]:
loader = data_loader('./config/dataset_config.json')
config = loader.get_config()

In [4]:
#set constants
dataset_name = "adult"
clustering_method = "k-centers"
num_experiments = 1
starting_randomState = 42
starting_num_clusters = 3
max_num_clusters = 20

In [5]:
X,_ = loader.load_dataset(dataset_name)
prepared_dataset = loader.prepare_dataset(X, dataset_name, complexity="simple")
validRandomStates = FairletExperiment.FindValidRandomStates(prepared_dataset, dataset_name, loader, num_experiments, starting_randomState)

In [6]:
sampled_dataset = loader.sample_data(prepared_dataset, dataset_name, validRandomStates[0])
reds, blues = loader.red_blue_split(sampled_dataset, dataset_name)

In [7]:
#mcf = MCF_decomposition(sampled_dataset, blues, reds, dataset_name, loader)
#mcf.MakeFairlets()


In [8]:
#mcf.fairlet_centers_dataset
#mcf.fairlet_center_sampled_dataset


In [9]:
k_center_instance = k_center(dataset_name, loader)
#mcf.CalculateClusterCostAndBalance(k_center_instance, None, clustering_method)

In [10]:
vanilla = Vanilla_decomposition(sampled_dataset, blues, reds, dataset_name, loader)
vanilla.makeFairlets(1, 2, validRandomStates[0])

208 fairlets have been identified.


0.2096858024597168

In [11]:
center_indices = vanilla.fairlet_centers

vanilla.dataset[vanilla.dataset.index.isin(center_indices)]

,sex,age,fnlwgt,education-num,capital-gain,hours-per-week
19484,0,47,244025,9,0,56
40834,1,43,282155,12,4650,40
27875,1,55,202220,9,2407,35
22608,0,34,261799,11,0,45
13398,0,50,173754,9,0,40
...,...,...,...,...,...,...
30338,0,33,119628,16,0,50
33592,0,25,203561,11,0,56
44161,0,19,179707,10,0,30
14740,1,20,110998,10,0,30


In [15]:
k_center_instance.fit(vanilla.fairlet_center_sampled_dataset, 3)
vanilla_f_costs, vanilla_f_cluster_mapping = k_center_instance.get_results()

In [16]:
fair=True
fairlet_information = vanilla.fairlets
fair_data = vanilla.fairlet_centers_dataset

In [22]:
fairlet_information

[[10423, 35881, 35859],
 [39667, 37602, 42053],
 [3150, 38250, 25056],
 [8393, 30037, 5931],
 [43465, 19685, 48029],
 [35387, 5545, 45771],
 [36399, 32838, 40687],
 [6478, 45442, 3714],
 [39168, 28269, 25132],
 [13506, 11348, 8180],
 [9743, 37304, 39628],
 [34399, 3168, 47793],
 [3025, 22975, 4817],
 [37869, 32185, 8853],
 [7096, 28895, 29186],
 [27722, 14825, 18503],
 [31255, 23406, 4],
 [28486, 43888, 41823],
 [31220, 708, 38864],
 [14339, 5712, 30520],
 [45097, 25539, 40740],
 [11601, 13643, 18579],
 [33772, 34580, 27943],
 [19949, 26388, 20078],
 [47101, 3211, 43348],
 [13040, 46435, 41201],
 [14172, 45727, 5525],
 [16729, 19484, 20495],
 [9086, 3467, 12865],
 [37039, 4252, 20139],
 [42945, 20211, 8871],
 [12653, 3085, 28911],
 [5568, 45971, 27133],
 [9501, 25166, 17697],
 [18627, 3792, 35762],
 [8034, 35003, 21546],
 [33993, 36853, 41369],
 [31606, 15035, 38469],
 [47176, 1599, 5187],
 [23014, 25922, 33802],
 [30667, 8784, 37536],
 [5803, 23020, 45204],
 [6202, 29598, 20540],
 [29

In [21]:
len(vanilla_f_cluster_mapping)

3

In [26]:
clusters = []
for center in vanilla_f_cluster_mapping.keys():
    clusters.append(vanilla.dataset.iloc[vanilla_f_cluster_mapping[center]])

    print(len(clusters))
    if fair==True:
        all_clustered_point_idx = []
        for cluster_point in clusters:
            clustered_points = cluster_point.index
            all_points = []
            for point in clustered_points:
                for fairlet_points in fairlet_information:
                    if point in fairlet_points:
                        all_points.append(fairlet_points)
            all_clustered_point_idx.append(all_points)

        all_clustered_points_lists = []
        for all_clustered_point in all_clustered_point_idx:
            all_clustered_points_lists.append([x for xs in all_clustered_point for x in xs])
    
        clusters = []
            for all_clustered_points_list in all_clustered_points_lists:
                clusters.append(self.dataset[self.dataset.index.isin(all_clustered_points_list)])

    balances = []
    for cluster in clusters:
        if len(cluster) > 1:
            balances.append(vanilla.mathHelper.get_balance(cluster, vanilla.config[vanilla.dataset_name]['sensitive_column'][0], vanilla.dataset_name))

1
[[29436, 7762, 21844, 46820, 23881, 30507, 21049, 5265, 12653, 3085, 28911, 16729, 19484, 20495, 28348, 44828, 40834, 48693, 1276, 47722, 12969, 22608, 27576, 31160, 21735, 36230, 43536, 46439, 45678, 18627, 3792, 35762, 35505, 9058, 2372, 16948, 16181, 3375, 36029, 5843, 31905, 28331, 48262, 11742, 147, 45613, 23288, 14182, 7134, 11429, 18957, 5503, 31256, 17112, 29642, 21247, 29350, 15298, 8489, 23638, 32710, 19387, 18067, 21063, 879, 7155, 44275, 19051, 9501, 25166, 17697, 44161, 21167, 332, 3136, 14740, 25588, 18597, 34254, 42906, 17657, 28326, 37254, 3928, 23848, 32526, 17593, 1422, 5711, 32507, 7841, 3253, 42939, 30150, 14172, 45727, 5525, 48262, 11742, 147, 34154, 43754, 10007, 29627, 26419, 47649, 17692, 42865, 43058, 31220, 708, 38864, 36388, 26338, 44348, 10803, 40051, 8406, 24304, 43953, 13370, 20882, 29640, 21581, 6478, 45442, 3714, 47176, 1599, 5187, 2868, 36271, 42010, 44840, 3030, 17370, 14617, 16714, 24735, 33592, 21190, 37616, 12208, 27700, 43761, 18115, 36862, 33401

In [ ]:
if fair==True:
        all_clustered_point_idx = []
        for cluster_point in clusters:
            clustered_points = cluster_point.index
            all_points = []
            for point in clustered_points:
                for coded_points_fairlet in coded_points_fairlets:
                    if point in coded_points_fairlet:
                        all_points.append(coded_points_fairlet)
            all_clustered_point_idx.append(all_points)

        all_clustered_points_lists = []
        for all_clustered_point in all_clustered_point_idx:
            all_clustered_points_lists.append([x for xs in all_clustered_point for x in xs])

        clusters = []
        for all_clustered_points_list in all_clustered_points_lists:
            clusters.append(vanilla.dataset[vanilla.dataset.index.isin(all_clustered_points_list)])

In [22]:
balances

[0.4766355140186916, 0.5547703180212014, 0]